# 開始安裝

In [ ]:
#20250828執行過
%pip install ipython-sql pandas sqlalchemy

In [ ]:
%load_ext sql

In [ ]:
import sqlite3
import pandas as pd

# 資料庫路徑（根據實際位置調整）
db_path = "../SQL/mysql/database/hi.db"

# 連接到 SQLite 資料庫
conn = sqlite3.connect(db_path)


# 練習區

## import內容和DB路經

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

## 查詢

In [ ]:
# 執行查詢並用 Pandas 顯示結果  
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

query = "SELECT * FROM Daily_report WHERE tag like '%sql%';"  
df = pd.read_sql(query, conn)  
df

## 插入單筆新資料

In [36]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

# 設定顯示選項 
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.width', 150)  
pd.set_option('display.max_colwidth', 60)
pd.set_option('display.expand_frame_repr', True) 
pd.set_option('display.colheader_justify', 'left')  

# # 建立 DataFrame
# df = pd.DataFrame({
#     'content': ['學習如何使用 SQLite 與 Python 進行資料庫操作'],
#     'reference': ['SQL/sqlite-python.ipynb'],
#     'tag': ['ipython-sql/sql/']
# })

# # 插入資料
# df.to_sql('Daily_report', conn, if_exists='append', index=False)

# 讀取並顯示最新插入的資料  
result = pd.read_sql_query("SELECT * FROM Daily_report ORDER BY id DESC LIMIT 3", conn)  # 讀取最新一筆資料
print(result)  


# 關閉連線
conn.close()


   id content                                   reference                tag               created_at          
0  9             學習如何使用 SQLite 與 Python 進行資料庫操作  SQL/sqlite-python.ipynb  ipython-sql/sql/  2025-08-28 14:46:34
1  8   失敗=>使用jupyter寫sqlite语法，改用python連接db並插入資料  SQL/sqlite-python.ipynb  ipython-sql/sql/  2025-08-28 14:28:42
2  7                         使用jupyter寫sqlite語法  SQL/sqlite-python.ipynb  ipython-sql/sql/  2025-08-28 14:23:39


In [ ]:
import pandas as pd
import sqlite3

# 建立連線
conn = sqlite3.connect('../SQL/mysql/database/hi.db')

# 讀取最新一筆資料
result = pd.read_sql_query("SELECT * FROM Daily_report ORDER BY id DESC LIMIT 1", conn)

# 手動左對齊顯示
def left_align_display(df):
    # 獲取每列最大寬度
    col_widths = {col: max(df[col].astype(str).str.len().max(), len(col)) for col in df.columns}
    
    # 打印表頭
    header = " ".join(f"{col:<{col_widths[col]}}" for col in df.columns)
    print(header)
    print("-" * len(header))
    
    # 打印資料
    for _, row in df.iterrows():
        row_str = " ".join(f"{str(row[col]):<{col_widths[col]}}" for col in df.columns)
        print(row_str)

# 顯示
left_align_display(result)


## 插入多筆資料

In [ ]:
import pandas as pd
import sqlite3

# 建立連線
db_path = "../SQL/mysql/database/hi.db"
conn = sqlite3.connect(db_path)

# 建立包含多筆資料的 DataFrame
df = pd.DataFrame({
    'content': [
        '使用jupyter寫sqlite语法', 
        '學習pandas插入sqlite', 
        'SQLite資料庫操作'
    ],
    'reference': [
        'SQL/sqlite-python.ipynb', 
        'pandas/sqlite-insert.ipynb', 
        'database/sqlite-tutorial.md'
    ],
    'tag': [
        'ipython-sql/sql/', 
        'pandas/database', 
        'tutorial/sqlite'
    ]
})

# 插入多筆資料
df.to_sql('Daily_report', conn, if_exists='append', index=False) # append表示新增資料，不覆蓋原有資料。index=False表示不插入DataFrame的索引欄位。

# 關閉連線
conn.close()
